# Tensorflow benchmark, yep
So, let's do it!

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
def matmul_flop(i,j,k):
    return (2*i*j*k - ik)

def bench()

batch_size = 128
l2_coef_np = np.float32(0.001)

num_hid = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  l2_coef = tf.constant(l2_coef_np)
  
  # Variables.
  #hidden layer
  w_hid = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_hid]))
  b_hid = tf.Variable(tf.zeros([num_hid]))
  #output layer  
  weights = tf.Variable(
    tf.truncated_normal([num_hid, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  out_hid = tf.nn.relu(tf.matmul(tf_train_dataset, w_hid) + b_hid)
  logits = tf.matmul(out_hid, weights) + biases
  loss = (tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    + l2_coef*tf.nn.l2_loss(weights) + l2_coef*tf.nn.l2_loss(w_hid) )
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss) 
                
    
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  v_out_hid = tf.nn.relu(tf.matmul(tf_valid_dataset, w_hid) + b_hid)
  v_logits = tf.matmul(v_out_hid, weights) + biases
  valid_prediction = tf.nn.softmax(v_logits)
  t_out_hid = tf.nn.relu(tf.matmul(tf_test_dataset, w_hid) + b_hid)
  t_logits = tf.matmul(t_out_hid, weights) + biases  
  test_prediction = tf.nn.softmax(t_logits)

In [ ]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 5000 == 0):
      print("Minibatch loss at step", step, ":", l)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))